In [1]:
import pandas as pd
import pickle
from scipy.stats import poisson

In [2]:
dict_table = pickle.load(open('dict_table', 'rb'))
df_historical_data = pd.read_csv('clean_fifa_worldcup_matches.csv')
df_fixture = pd.read_csv('clean_fifa_worldcup_fixture.csv')

### 1 Calcular Team Strenght

In [3]:
# dividir df en df_home and df_away
df_home = df_historical_data[['HomeTeam', 'HomeGoals', 'AwayGoals']]
df_away = df_historical_data[['AwayTeam', 'HomeGoals', 'AwayGoals']]

In [4]:
# renombrar columnas
df_home = df_home.rename(columns={'HomeTeam': 'Team', 'HomeGoals': 'GoalsScored', 'AwayGoals':'GoalsConceded'})
df_away = df_away.rename(columns={'AwayTeam': 'Team', 'HomeGoals': 'GoalsConceded', 'AwayGoals':'GoalsScored'})

In [5]:
# concatenar df_home y df_away, hacer group por team y calcular promedio
df_team_strength = pd.concat([df_home, df_away], ignore_index = True).groupby('Team').mean()
df_team_strength

,GoalsScored,GoalsConceded
Team,,
Algeria,1.000000,1.461538
Angola,0.333333,0.666667
Argentina,1.691358,1.148148
Australia,0.812500,1.937500
Austria,1.482759,1.620690
...,...,...
Uruguay,1.553571,1.321429
Wales,0.800000,0.800000
West Germany,2.112903,1.241935


### 2 Funcion predict_points 

In [6]:
def predict_points(home, away):
    if ((home in df_team_strength.index) and (away in df_team_strength.index)):
        # goals_scored * goals_conceded
        lamb_home = df_team_strength.at[home, 'GoalsScored'] * df_team_strength.at[away, 'GoalsConceded']
        lamb_away = df_team_strength.at[away, 'GoalsScored'] * df_team_strength.at[home, 'GoalsConceded']
        prob_home, prob_away, prob_draw = 0, 0, 0

        for x in range(0,11): #number of goals away team
            for y in range(0, 11): #number of goals away team
                p = poisson.pmf(x, lamb_home) * poisson.pmf(y, lamb_away)
                if x == y:
                    prob_draw += p
                elif x > y:
                    prob_home += p
                else:
                    prob_away += p

        points_home = 3 * prob_home + prob_draw
        points_away = 3 * prob_away + prob_draw
        return (points_home, points_away)

    else:
        return (0, 0)

#### 2.1 Testear Funcion

In [7]:
# Testear con partidos: Argentina - Mexico, England - United States
predict_points('Argentina', 'France')

(1.3318238390626087, 1.463097021203372)

### 3 Prediccicion del Mundial

#### 3.1 Fase de Grupos

In [8]:
# dividiendo fixture en grupo, octavos, cuartos,...
df_fixture_group_48 = df_fixture[:48].copy()
df_fixture_knockout = df_fixture[48:56].copy()
df_fixture_quarter = df_fixture[56:60].copy()
df_fixture_semi = df_fixture[60:62].copy()
df_fixture_final = df_fixture[62:].copy()

In [9]:
df_fixture_group_48[df_fixture_group_48['home'].isin(['Argentina', 'Saudi Arabia', 'Mexico', 'Poland'])]

,home,score,away,year
12,Argentina,Match 8,Saudi Arabia,2022
13,Mexico,Match 7,Poland,2022
14,Poland,Match 22,Saudi Arabia,2022
15,Argentina,Match 24,Mexico,2022
16,Poland,Match 39,Argentina,2022
17,Saudi Arabia,Match 40,Mexico,2022


In [10]:
dict_table['Group C'].loc[dict_table['Group C']['Team'] == 'Argentina', 'Pts']

0    0
Name: Pts, dtype: int64

In [11]:
# Correr todos los partidos de la fase de grupos y actualizar las tablas de cada grupo
for group in dict_table:
    teams_in_group = dict_table[group]['Team'].values
    df_fixture_group_6 = df_fixture_group_48[df_fixture_group_48['home'].isin(teams_in_group)]

    for index, row in df_fixture_group_6.iterrows():
        home, away = row['home'], row['away']
        points_home, points_away = predict_points(home, away)

        dict_table[group].loc[dict_table[group]['Team'] == home, 'Pts'] += points_home
        dict_table[group].loc[dict_table[group]['Team'] == away, 'Pts'] += points_away

    dict_table[group] = dict_table[group].sort_values('Pts', ascending = False).reset_index()
    dict_table[group] = dict_table[group][['Team', 'Pts']]
    dict_table[group] = dict_table[group].round(0)

In [12]:
# mostrar tabla actualizada
for group in dict_table:
    print(dict_table[group])

          Team  Pts
0  Netherlands  4.0
1      Senegal  2.0
2      Ecuador  2.0
3    Qatar (H)  0.0
            Team  Pts
0        England  6.0
1          Wales  5.0
2  United States  3.0
3           Iran  2.0
           Team  Pts
0     Argentina  7.0
1        Poland  6.0
2        Mexico  4.0
3  Saudi Arabia  1.0
        Team  Pts
0     France  7.0
1    Denmark  6.0
2    Tunisia  3.0
3  Australia  2.0
         Team  Pts
0     Germany  7.0
1       Spain  5.0
2       Japan  3.0
3  Costa Rica  2.0
      Team  Pts
0  Croatia  7.0
1  Belgium  6.0
2  Morocco  4.0
3   Canada  0.0
          Team  Pts
0       Brazil  8.0
1  Switzerland  4.0
2       Serbia  3.0
3     Cameroon  2.0
          Team  Pts
0     Portugal  6.0
1      Uruguay  5.0
2        Ghana  4.0
3  South Korea  2.0


### 3.2 Octavos

In [13]:
df_fixture_knockout

,home,score,away,year
48,Winners Group A,Match 49,Runners-up Group B,2022
49,Winners Group C,Match 50,Runners-up Group D,2022
50,Winners Group D,Match 52,Runners-up Group C,2022
51,Winners Group B,Match 51,Runners-up Group A,2022
52,Winners Group E,Match 53,Runners-up Group F,2022
53,Winners Group G,Match 54,Runners-up Group H,2022
54,Winners Group F,Match 55,Runners-up Group E,2022
55,Winners Group H,Match 56,Runners-up Group G,2022


In [14]:
# actualizar el fixture de octavos con el 1 puesto (group winner) y 2 puesto (runners up)
for group in dict_table:
    group_winner = dict_table[group].loc[0, 'Team']
    runners_up = dict_table[group].loc[1, 'Team']

    df_fixture_knockout.replace({f'Winners {group}': group_winner,
                            f'Runners-up {group}': runners_up}, inplace = True)

df_fixture_knockout['winner'] = '?'
df_fixture_knockout

,home,score,away,year,winner
48,Netherlands,Match 49,Wales,2022,?
49,Argentina,Match 50,Denmark,2022,?
50,France,Match 52,Poland,2022,?
51,England,Match 51,Senegal,2022,?
52,Germany,Match 53,Belgium,2022,?
53,Brazil,Match 54,Uruguay,2022,?
54,Croatia,Match 55,Spain,2022,?
55,Portugal,Match 56,Switzerland,2022,?


In [15]:
# crear funcion get_winner
def get_winner(df_fixture_update):
    for index, row in df_fixture_update.iterrows():
        home, away = row['home'], row['away']
        points_home, points_away = predict_points(home, away)

        if points_home > points_away:
            winner = home
        else:
            winner = away

        df_fixture_update.loc[index, 'winner'] = winner
    return df_fixture_update

In [16]:
get_winner(df_fixture_knockout)

,home,score,away,year,winner
48,Netherlands,Match 49,Wales,2022,Netherlands
49,Argentina,Match 50,Denmark,2022,Argentina
50,France,Match 52,Poland,2022,France
51,England,Match 51,Senegal,2022,England
52,Germany,Match 53,Belgium,2022,Germany
53,Brazil,Match 54,Uruguay,2022,Brazil
54,Croatia,Match 55,Spain,2022,Spain
55,Portugal,Match 56,Switzerland,2022,Portugal


### 3.3 Cuartos de Final

In [17]:
df_fixture_knockout

,home,score,away,year,winner
48,Netherlands,Match 49,Wales,2022,Netherlands
49,Argentina,Match 50,Denmark,2022,Argentina
50,France,Match 52,Poland,2022,France
51,England,Match 51,Senegal,2022,England
52,Germany,Match 53,Belgium,2022,Germany
53,Brazil,Match 54,Uruguay,2022,Brazil
54,Croatia,Match 55,Spain,2022,Spain
55,Portugal,Match 56,Switzerland,2022,Portugal


In [18]:
df_fixture_quarter

,home,score,away,year
56,Winners Match 53,Match 58,Winners Match 54,2022
57,Winners Match 49,Match 57,Winners Match 50,2022
58,Winners Match 55,Match 60,Winners Match 56,2022
59,Winners Match 51,Match 59,Winners Match 52,2022


In [19]:
def update_table(df_fixture_round_1, df_fixture_round_2):
    for index, row in df_fixture_round_1.iterrows():
        winner = df_fixture_round_1.loc[index, 'winner']
        match = df_fixture_round_1.loc[index, 'score']
        df_fixture_round_2.replace({f'Winners {match}': winner}, inplace = True)

    df_fixture_round_2['winner'] = '?'
    return df_fixture_round_2

In [20]:
update_table(df_fixture_knockout, df_fixture_quarter)

,home,score,away,year,winner
56,Germany,Match 58,Brazil,2022,?
57,Netherlands,Match 57,Argentina,2022,?
58,Spain,Match 60,Portugal,2022,?
59,England,Match 59,France,2022,?


In [21]:
df_fixture_quarter

,home,score,away,year,winner
56,Germany,Match 58,Brazil,2022,?
57,Netherlands,Match 57,Argentina,2022,?
58,Spain,Match 60,Portugal,2022,?
59,England,Match 59,France,2022,?


In [22]:
get_winner(df_fixture_quarter)

,home,score,away,year,winner
56,Germany,Match 58,Brazil,2022,Brazil
57,Netherlands,Match 57,Argentina,2022,Netherlands
58,Spain,Match 60,Portugal,2022,Portugal
59,England,Match 59,France,2022,France


In [23]:
df_fixture_quarter

,home,score,away,year,winner
56,Germany,Match 58,Brazil,2022,Brazil
57,Netherlands,Match 57,Argentina,2022,Netherlands
58,Spain,Match 60,Portugal,2022,Portugal
59,England,Match 59,France,2022,France


### 3.4 Semifinal

In [24]:
update_table(df_fixture_quarter, df_fixture_semi)

,home,score,away,year,winner
60,Netherlands,Match 61,Brazil,2022,?
61,France,Match 62,Portugal,2022,?


In [25]:
df_fixture_semi

,home,score,away,year,winner
60,Netherlands,Match 61,Brazil,2022,?
61,France,Match 62,Portugal,2022,?


In [26]:
get_winner(df_fixture_semi)

,home,score,away,year,winner
60,Netherlands,Match 61,Brazil,2022,Brazil
61,France,Match 62,Portugal,2022,France


In [27]:
df_fixture_semi

,home,score,away,year,winner
60,Netherlands,Match 61,Brazil,2022,Brazil
61,France,Match 62,Portugal,2022,France


### 3.5 Final

In [28]:
update_table(df_fixture_semi, df_fixture_final)

,home,score,away,year,winner
62,Losers Match 61,Match 63,Losers Match 62,2022,?
63,Brazil,Match 64,France,2022,?


In [29]:
df_fixture_final

,home,score,away,year,winner
62,Losers Match 61,Match 63,Losers Match 62,2022,?
63,Brazil,Match 64,France,2022,?


In [30]:
get_winner(df_fixture_final)

,home,score,away,year,winner
62,Losers Match 61,Match 63,Losers Match 62,2022,Losers Match 62
63,Brazil,Match 64,France,2022,Brazil


In [31]:
df_fixture_final

,home,score,away,year,winner
62,Losers Match 61,Match 63,Losers Match 62,2022,Losers Match 62
63,Brazil,Match 64,France,2022,Brazil
